### Actividad: "Médicos vs. Máquinas" 

Esta dinámica convierte el análisis de Machine Learning en un juego 
interactivo donde los alumnos compiten entre sí y contra el modelo de IA para 
diagnosticar diabetes.

### Cómo funciona 

 Fase 1: "Doctor por un día"  
- Se seleccionan 5-10 muestras de pacientes del dataset sin mostrar si 
tienen diabetes. 
- Cada equipo (o alumno) analiza los valores (glucosa, IMC, presión, etc.) 
y decide si el paciente tiene o no diabetes. 
- Se anotan las predicciones en una pizarra o tabla.

**Objetivo**: Poner a prueba el razonamiento humano en comparación con el 
modelo. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_selection import RFECV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

In [2]:
columnas = ["Embarazos", "Glucosa", "Presion_Arterial", "Espesor_Piel", 
"Insulina", "IMC", "DiabetesPedigree", "Edad", "Diabetes"]
# Cargamos los datos
pima = pd.read_csv(url, names=columnas)

In [3]:
# Inspeccionar las primeras filas
pima.head(10)

,Embarazos,Glucosa,Presion_Arterial,Espesor_Piel,Insulina,IMC,DiabetesPedigree,Edad,Diabetes
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


In [4]:
# Obtener estadísticas descriptivas (media, desviación estándar, valores faltantes)
pima.describe()

,Embarazos,Glucosa,Presion_Arterial,Espesor_Piel,Insulina,IMC,DiabetesPedigree,Edad,Diabetes
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [5]:
# Revisar valores faltantes
pima.isnull().sum()

Embarazos           0
Glucosa             0
Presion_Arterial    0
Espesor_Piel        0
Insulina            0
IMC                 0
DiabetesPedigree    0
Edad                0
Diabetes            0
dtype: int64

In [6]:
pima.columns

Index(['Embarazos', 'Glucosa', 'Presion_Arterial', 'Espesor_Piel', 'Insulina',
       'IMC', 'DiabetesPedigree', 'Edad', 'Diabetes'],
      dtype='object')

In [7]:
# Crear nueva columna de Estado nutricional
Estado_nutricional = pd.Series([]) 


In [8]:
# Estado nutricional basado en el IMC

for i in range(len(pima)): 
    if pima['IMC'][i] == 0.0: 
        Estado_nutricional[i]="NA"
    
    elif pima['IMC'][i] < 18.5: 
        Estado_nutricional[i]="Bajo peso"
        
    elif pima['IMC'][i] < 25: 
        Estado_nutricional[i]="Normal"
  
    elif pima['IMC'][i] >= 25 and pima['IMC'][i] < 30: 
        Estado_nutricional[i]="Sobre peso"
  
    elif pima['IMC'][i] >= 30: 
        Estado_nutricional[i]="Obeso"
        
    else: 
        Estado_nutricional[i]= pima['IMC'][i] 

In [9]:
# Insertamos la nueva columna Estado nutricional

pima.insert(6, "Estado Nutricional", Estado_nutricional)

In [10]:
pima.head()

,Embarazos,Glucosa,Presion_Arterial,Espesor_Piel,Insulina,IMC,Estado Nutricional,DiabetesPedigree,Edad,Diabetes
0,6,148,72,35,0,33.6,Obeso,0.627,50,1
1,1,85,66,29,0,26.6,Sobre peso,0.351,31,0
2,8,183,64,0,0,23.3,Normal,0.672,32,1
3,1,89,66,23,94,28.1,Sobre peso,0.167,21,0
4,0,137,40,35,168,43.1,Obeso,2.288,33,1


In [11]:
pima['Estado Nutricional'].value_counts()

Estado Nutricional
Obeso         472
Sobre peso    179
Normal        102
NA             11
Bajo peso       4
Name: count, dtype: int64

In [12]:
# Crear OGTT_Interpretacion (Interpretación de niveles de Glucosa) columna
OGTT_Interpretacion = pd.Series([])

In [13]:
# Interpretación de niveles de Glucosa utilizando OGTT levels
for i in range(len(pima)): 
    if pima['Glucosa'][i] == 0.0: 
        OGTT_Interpretacion [i]="NA"
    
    elif pima['Glucosa'][i] <= 140: 
        OGTT_Interpretacion [i]="Normal"
    
    elif pima['Glucosa'][i] > 140 & pima['Glucosa'][i] <= 198: 
        OGTT_Interpretacion [i]="Intolerancia a la glucosa"
        
    elif pima['Glucosa'][i] > 198: 
        OGTT_Interpretacion[i]="Nivel diabético"
        
    else: 
        OGTT_Interpretacion [i]= pima['Glucosa'][i] 

In [14]:
# Insertamos la nueva columna OGTT_Interpretacion
pima.insert(8, "Resultado de Glucosa", OGTT_Interpretacion)

In [15]:
pima.head()

,Embarazos,Glucosa,Presion_Arterial,Espesor_Piel,Insulina,IMC,Estado Nutricional,DiabetesPedigree,Resultado de Glucosa,Edad,Diabetes
0,6,148,72,35,0,33.6,Obeso,0.627,Intolerancia a la glucosa,50,1
1,1,85,66,29,0,26.6,Sobre peso,0.351,Normal,31,0
2,8,183,64,0,0,23.3,Normal,0.672,Intolerancia a la glucosa,32,1
3,1,89,66,23,94,28.1,Sobre peso,0.167,Normal,21,0
4,0,137,40,35,168,43.1,Obeso,2.288,Normal,33,1


In [16]:
pima['Resultado de Glucosa'].value_counts()

Resultado de Glucosa
Normal                       571
Intolerancia a la glucosa    192
NA                             5
Name: count, dtype: int64

In [17]:
# Guardamos los datos en un nuevo archivo
pima.to_csv('datos_con_diabetes.csv', index=False)